In [1]:
from qiskit import IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus
from qiskit.providers.ibmq import least_busy
import numpy as np
from helpers import *
DIR = 'TEST_DATA/'
getPath = lambda fn : DIR+fn

In [2]:
qps = createModelsFromDir(DIR)
# solve qps with Minimum Eigen Optimizer

In [3]:
qp = qps['test_2']
qubo = QuadraticProgramToQubo().convert(qp)
qubo.to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: test_2
// single vars section
dvar bool x0_0;
dvar bool x0_1;
dvar bool A0_leb0@int_slack@0;

minimize
 108 x0_0 - 180 x0_1 + 18 A0_leb0@int_slack@0 [ 36 x0_0^2 - 120 x0_0*x0_1
 + 12 x0_0*A0_leb0@int_slack@0 + 100 x0_1^2 - 20 x0_1*A0_leb0@int_slack@0
 + A0_leb0@int_slack@0^2 ] + 81;
 
subject to {

}


In [4]:
IBMQ.load_account() 
provider = IBMQ.get_provider(hub='ibm-q')
backends = provider.backends(filters=lambda x: x.configuration().n_qubits >= qubo.get_num_vars()
                                     and not x.configuration().simulator
                                    and x.status().operational==True)

print(f" Backend with only {least_busy(backends).status().pending_jobs} pending jobs is {least_busy(backends).name()}")

/home/tom/anaconda3/envs/Qiskit_Ocean/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


 Backend with only 108 pending jobs is ibmq_lima


In [5]:
backend = provider.get_backend('ibmq_lima')
quantum_instance = QuantumInstance(backend)
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=quantum_instance))

In [ ]:
res = qaoa.solve(qp)
print(res)
# expected to run in 7 hours...

# Produced Jobs in IBMQ
60434c065f0ef5a3e08ee68f on ibmq_belem cancelled after several hours of waiting...



In [ ]:
cres  = qp.model.solve()
print(cres)